In [165]:
# Install necessary libraries
#!pip install -q huggingface_hub datasets tqdm
#!pip install fuzzywuzzy
#!pip install python-Levenshtein

In [166]:
import pandas as pd
import numpy as np
import spacy
import re
import time
import os
from huggingface_hub import InferenceClient
from tqdm.auto import tqdm
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import ast
from google.colab import userdata
#nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

### Data Loading and Initial Preprocessing

In [167]:
resumeatlas_path = "resumeAtlas.csv"
df = pd.read_csv(resumeatlas_path)
print(df.head())

     Category                                               Text
0  Accountant  education omba executive leadership university...
1  Accountant  howard gerrard accountant deyjobcom birmingham...
2  Accountant  kevin frank senior accountant inforesumekraftc...
3  Accountant  place birth nationality olivia ogilvy accounta...
4  Accountant  stephen greet cpa senior accountant 9 year exp...


In [168]:
df["Text_Clean"] = df["Text"].str.lower()
df["Text_Clean"] = df["Text_Clean"].str.replace(r'\d+', ' ', regex=True)
df["Text_Clean"] = df["Text_Clean"].str.replace(r'[^\w\s]', ' ', regex=True)
df["Text_Clean"] = df["Text_Clean"].str.replace(r'\s+', ' ', regex=True)
df["Text_Clean"] = df["Text_Clean"].str.strip()

In [169]:
# Step 1: Create clean column with lowercase
df["Text_Clean"] = df["Text"].str.lower()


In [170]:
# Step 2: Remove numbers/dates - simplified approach
df["Text_Clean"] = df["Text_Clean"].str.replace(r'\d+', ' ', regex=True)

In [171]:
# Step 3: Remove special chars and normalize spaces
df["Text_Clean"] = df["Text_Clean"].str.replace(r'[^\w\s]', ' ', regex=True)
df["Text_Clean"] = df["Text_Clean"].str.replace(r'\s+', ' ', regex=True)

In [172]:
# Step 4: Final cleanup
df["Text_Clean"] = df["Text_Clean"].str.strip()

In [173]:
print("Original:\n", repr(df.loc[0, "Text"]))
print("\nCleaned:\n", repr(df.loc[0, "Text_Clean"]))

Original:
 'education omba executive leadership university texas 20162018 bachelor science accounting richland college 20052008 training certifications certified management accountant cma certified financial modeling valuation analyst compliance antimoney laundering 092016 american institute banking certified public account cpa lean six sigma green belt certified trade products financial regulations 082016 american institute banking achievements speaker bringing leader within 082019 successfully presented empowering speech leadership 500 participants speaker dallas convention cpas 032019 successfully delivered seminar 3k cpas convention guests teaching experience online teacher udemy 2017 taught online accounting nonaccountant course udemy similar online teaching platforms developed effective teaching modules materials curriculum target students took feedbacks students assist improving teaching methodology materials professional memberships affiliations american society executives 2018

In [174]:
df.to_csv("resumeAtlas_cleaned.csv", index=False)

In [175]:
df = pd.read_csv("resumeAtlas_cleaned.csv")
df.head()

,Category,Text,Text_Clean
0,Accountant,education omba executive leadership university...,education omba executive leadership university...
1,Accountant,howard gerrard accountant deyjobcom birmingham...,howard gerrard accountant deyjobcom birmingham...
2,Accountant,kevin frank senior accountant inforesumekraftc...,kevin frank senior accountant inforesumekraftc...
3,Accountant,place birth nationality olivia ogilvy accounta...,place birth nationality olivia ogilvy accounta...
4,Accountant,stephen greet cpa senior accountant 9 year exp...,stephen greet cpa senior accountant year exper...


### Cleaning and Extracting ( LLM only )

In [176]:

#  Initial Setup: Hugging Face Token and InferenceClient
HF_TOKEN = userdata.get('HF_TOKEN_L')
client = InferenceClient(
    provider="sambanova",
    api_key=HF_TOKEN,
)

#  Load Input Resumes
df = pd.read_csv("resumeAtlas_cleaned.csv")
df_resumes_for_extraction = df[df['Category'] == 'Human Resources'].copy()

RESUME_TEXT_COLUMN = "Text_Clean"

def clean_output(text):
    if not isinstance(text, str) or not text.strip():
        return ""

# Convert to lowercase and split by common delimiters
    skills_list = [s.strip().lower() for s in re.split(r"[\s,;]+", text) if s.strip()]
    return ", ".join(skills_list) # Return as string for raw column


# Define the Adapted Skill Extraction Function using InferenceClient
LLAMA_MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

def extract_skills_llama3_sambanova(resume_text):
    MAX_RESUME_CHARS_FOR_LLM = 4000
    if len(resume_text) > MAX_RESUME_CHARS_FOR_LLM:
        resume_text_for_llm = resume_text[:MAX_RESUME_CHARS_FOR_LLM] + "..."
    else:
        resume_text_for_llm = resume_text

    messages = [
        {"role": "system", "content": "You are a professional resume parser. Extract all technical skills from the resume below. Only output a clean, comma-separated list. No extra words."},
        {"role": "user", "content": f"{resume_text_for_llm}\n\nSkills:"}
    ]

    completion = client.chat.completions.create(
        model=LLAMA_MODEL_ID,
        messages=messages,
        temperature=0.2,
        max_tokens=200,
    )
    raw_output = completion.choices[0].message.content
    return raw_output

# Apply the Skill Extraction Function
tqdm.pandas(desc=f"Extracting Skills ({LLAMA_MODEL_ID} via SambaNova)")
df_resumes_for_extraction["extracted_skills_raw"] = df_resumes_for_extraction[RESUME_TEXT_COLUMN].progress_apply(extract_skills_llama3_sambanova)

output_path = "extracted_skills_output.csv"
df_resumes_for_extraction.to_csv(output_path, index=False)

Extracting Skills (meta-llama/Llama-3.3-70B-Instruct via SambaNova):   0%|          | 0/360 [00:00<?, ?it/s]

In [177]:
df = pd.read_csv("extracted_skills_output.csv")
print(df.head())

          Category                                               Text  \
0  Human Resources  amelia quinton 9835 water street lake zurich i...   
1  Human Resources  sunidhi badruka mobile xxxxxxxxxx email pbadru...   
2  Human Resources  robert wales dubai united arab emirates 050 00...   
3  Human Resources  jane sample 123 main street edmonton ab t5k 2y...   
4  Human Resources  james h maxwell sphr 1876 nightvale path carso...   

                                          Text_Clean  \
0  amelia quinton water street lake zurich il aqu...   
1  sunidhi badruka mobile xxxxxxxxxx email pbadru...   
2  robert wales dubai united arab emirates mobile...   
3  jane sample main street edmonton ab t k y info...   
4  james h maxwell sphr nightvale path carson il ...   

                                extracted_skills_raw  
0  Microsoft Access, database management, client ...  
1  Recruitment, Sourcing, Performance Management,...  
2  Compensation, Benefits, Resourcing, Organizati...  
3  M

### Mapping skills to ESCO

In [178]:
df_mapping = pd.read_csv("extracted_skills_output.csv")
esco_df = pd.read_csv("skills_en.csv")[['preferredLabel', 'altLabels']].copy()

# Clean LLM Output Function
def clean_extracted_skills(text):
    """Cleans raw LLM text output into a set of normalized skill phrases."""
    if not isinstance(text, str) or not text.strip():
        return set()

    text = text.lower().replace(';', ',').replace(' and ', ', ')
    raw_skill_candidates = re.split(r',\s*|\s*,\s*', text)

    return {
        re.sub(r'[^a-z0-9\s\.\+#-]+', '', s).strip()
        for s in raw_skill_candidates
        if s.strip() and len(s.strip()) > 1 and not s.strip().isdigit()
    }

# Apply cleaning
df_mapping['extracted_skills_set'] = df_mapping['extracted_skills_raw'].apply(clean_extracted_skills)

# Pre-process ESCO data for faster lookups
esco_preferred_lower_list = esco_df['preferredLabel'].str.lower().tolist()
pref_map = {label.lower(): label for label in esco_df['preferredLabel'].unique()}

alt_map = {
    alt.strip(): row['preferredLabel']
    for _, row in esco_df.iterrows()
    if isinstance(row['altLabels'], str)
    for alt in row['altLabels'].lower().replace('"', '').split('|')
    if alt.strip()
}

# ESCO Mapping Function
def map_to_esco(skill_name, pref_map_ref, alt_map_ref, esco_lower_list_ref):
    """Maps an extracted skill to an ESCO preferredLabel using exact/fuzzy matching."""
    skill = str(skill_name).lower().strip()
    if not skill:
        return None

    # Exact match (preferred or alternative)
    if skill in pref_map_ref: return pref_map_ref[skill]
    if skill in alt_map_ref: return alt_map_ref[skill]

    # Fuzzy match
    fuzzy_match_result = process.extractOne(skill, esco_lower_list_ref, scorer=fuzz.token_set_ratio)
    if fuzzy_match_result and fuzzy_match_result[1] >= 60: # score >= 60
        # Retrieve original preferredLabel case
        original_label_row = esco_df[esco_df['preferredLabel'].str.lower() == fuzzy_match_result[0]]
        if not original_label_row.empty:
            return original_label_row['preferredLabel'].iloc[0]
    return None

# Apply ESCO Mapping
df_mapping["mapped_skills_str"] = df_mapping['extracted_skills_set'].progress_apply(
    lambda s: ", ".join(sorted(list(filter(None, [map_to_esco(x, pref_map, alt_map, esco_preferred_lower_list) for x in s]))))
)

output_path = "mapped_skills_output.csv"

columns_to_save = [
    'Unnamed: 0', 'Category', 'Text_Clean', 'extracted_skills_raw',
    'original_extracted_skills_for_eval', 'mapped_skills_str'
]
df_mapping['original_extracted_skills_for_eval'] = df_mapping['extracted_skills_set'].apply(lambda s: str(list(s)))

existing_columns_to_save = [col for col in columns_to_save if col in df_mapping.columns]

df_mapping[existing_columns_to_save].to_csv(output_path, index=False)

print(f"Mapped skills saved to {output_path}")

Extracting Skills (meta-llama/Llama-3.3-70B-Instruct via SambaNova):   0%|          | 0/360 [00:00<?, ?it/s]

Mapped skills saved to mapped_skills_output.csv


In [179]:
df = pd.read_csv("mapped_skills_output.csv")
df.head()

,Category,Text_Clean,extracted_skills_raw,original_extracted_skills_for_eval,mapped_skills_str
0,Human Resources,amelia quinton water street lake zurich il aqu...,"Microsoft Access, database management, client ...","['microsoft access', 'client tracking', 'food ...","ATM systems, ATM systems, Agile development, I..."
1,Human Resources,sunidhi badruka mobile xxxxxxxxxx email pbadru...,"Recruitment, Sourcing, Performance Management,...","['payroll management', 'new hire feedback sess...",adopt ways to foster biodiversity and animal w...
2,Human Resources,robert wales dubai united arab emirates mobile...,"Compensation, Benefits, Resourcing, Organizati...","['career ladders', 'human resource development...","Agile development, adhere to organisational gu..."
3,Human Resources,jane sample main street edmonton ab t k y info...,"MS Word, Excel, PowerPoint, Great Plains softw...","['customer service', 'scheduling', 'time manag...","Agile development, CAM software, CAM software,..."
4,Human Resources,james h maxwell sphr nightvale path carson il ...,"Human Capital Management, Talent Acquisition, ...","['national employment law', 'diversity', 'work...","STAF, animal acquisition, animal acquisition, ..."


### ESCO Ground Truth

In [180]:
# Load ESCO datasets
occupations = pd.read_csv('occupations_en.csv')
occ_skill_rels = pd.read_csv('occupationSkillRelations_en.csv')
skills = pd.read_csv('skills_en.csv')

# Prepare job data
jobs = pd.DataFrame({
    'job_id': [1],
    'category': ['Human Resources Manager'],
    'other_info': ['...']
})

# Map to ESCO occupations
jobs['norm_category'] = jobs['category'].str.lower().str.strip()
occupations['norm_label'] = occupations['preferredLabel'].str.lower().str.strip()

# Merge jobs with ESCO occupations and skills
esco_skills = (
    jobs.merge(
        occupations[['conceptUri', 'preferredLabel', 'norm_label']],
        left_on='norm_category',
        right_on='norm_label',
        how='left'
    )
    .merge(
        occ_skill_rels,
        left_on='conceptUri',
        right_on='occupationUri',
        how='left'
    )
    .merge(
        skills[['conceptUri', 'preferredLabel']],
        left_on='skillUri',
        right_on='conceptUri',
        how='left'
    )
)

# Aggregate expected skills
expected_skills = (
    esco_skills.dropna(subset=['preferredLabel_y'])
    .groupby(['job_id', 'category'], as_index=False)
    ['preferredLabel_y'].agg(list)
    .rename(columns={'preferredLabel_y': 'expected_esco_skills'})
)

expected_skills.to_csv('expected_esco_skills.csv', index=False)
pd.read_csv("expected_esco_skills.csv").head()

,job_id,category,expected_esco_skills
0,1,Human Resources Manager,"['labour legislation', 'outplacement', 'employ..."


### Merge Expected ESCO Skills for LLM Evaluation

In [181]:
df_map = pd.read_csv("mapped_skills_output.csv")
df_exp = pd.read_csv("expected_esco_skills.csv")

# Clean the category names for consistent merging
df_map['Category'] = df_map['Category'].str.lower().str.strip()
df_exp['category'] = df_exp['category'].str.lower().str.strip()

# Get the expected ESCO skills for Human Resources Manager
human_resources_skills = df_exp[df_exp['category'] == 'human resources manager']['expected_esco_skills'].iloc[0]

# Add expected skills column to all Human Resources Manager rows
df_map['expected_esco_skills'] = df_map['Category'].apply(
    lambda x: human_resources_skills if x == 'human resources' else None
)

# Reorder columns for better readability
final_df = df_map[['Category', 'expected_esco_skills', 'extracted_skills_raw',
                'original_extracted_skills_for_eval', 'mapped_skills_str']]

print(final_df.head().to_markdown(index=False))
final_df.to_csv("mapped_skills_with_expected_esco_for_hr.csv", index=False)

| Category        | expected_esco_skills                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

### Performance Evaluation LLM

In [182]:
df = pd.read_csv("mapped_skills_with_expected_esco_for_hr.csv")

def parse_skills(skills_str):
    """Convert skill string to standardized set of skills."""
    if pd.isna(skills_str) or not str(skills_str).strip():
        return set()
    try:
        return {str(s).lower().strip() for s in ast.literal_eval(skills_str)}
    except (ValueError, SyntaxError):
        return {s.strip().lower() for s in str(skills_str).split(',') if s.strip()}

# Process skill columns
df['expected_skills'] = df['expected_esco_skills'].apply(parse_skills)
df['mapped_skills'] = df['mapped_skills_str'].apply(parse_skills)

def calculate_metrics(gt_set, pred_set):
    """Calculate precision, recall, and F1-score between skill sets."""
    tp = len(pred_set & gt_set)
    fp = len(pred_set - gt_set)
    fn = len(gt_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Filter and calculate metrics for human resources
ce_df = df[df['Category'].str.lower() == 'human resources'].copy()

if ce_df.empty:
    print("No human resources records found with expected skills.")
else:
    # Calculate metrics
    ce_df[['precision', 'recall', 'f1_score']] = ce_df.apply(
        lambda r: calculate_metrics(r['expected_skills'], r['mapped_skills']),
        axis=1, result_type='expand'
    )

    # Add intersection skills
    ce_df['intersection_skills'] = ce_df.apply(
        lambda r: list(r['mapped_skills'] & r['expected_skills']), axis=1
    )

    display_cols = ['Category', 'intersection_skills','precision',
                    'recall', 'f1_score', 'mapped_skills_str','expected_esco_skills']
    print(ce_df[display_cols].head(10).to_markdown(index=False))
    print(f"\nAverage Precision: {ce_df['precision'].mean():.4f}")
    print(f"Average Recall:    {ce_df['recall'].mean():.4f}")
    print(f"Average F1-Score:  {ce_df['f1_score'].mean():.4f}")

| Category        | intersection_skills                                                                                                               |   precision |     recall |   f1_score | mapped_skills_str                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

### Cleaning and Extracting ( LLM +KG )

In [183]:
df_resumes_for_extraction = pd.read_csv("resumeAtlas_cleaned.csv")
df_resumes_for_extraction = df_resumes_for_extraction[df_resumes_for_extraction['Category'] == 'Human Resources'].copy()
RESUME_TEXT_COLUMN = "Text_Clean"

# Load ESCO data for Domain-Specific Skills
esco_df_full = pd.read_csv("skills_en.csv")

# human resources Keywords
hr_keywords = [
    "Recruitment", "Talent Acquisition", "Onboarding", "Offboarding",
    "Employee Relations", "Labor Relations", "Performance Management",
    "Compensation", "Benefits Administration", "Payroll",
    "Training and Development", "Succession Planning", "Workforce Planning",
    "HRIS", "Workday", "SAP SuccessFactors", "Oracle HCM",
    "BambooHR", "ADP", "Paychex", "UltiPro", "PeopleSoft",
    "Applicant Tracking Systems", "ATS", "Talent Management Systems",
    "Employment Law", "Labor Law", "EEO Compliance", "ADA Compliance",
    "FMLA", "Workers Compensation", "OSHA Compliance", "HR Compliance",
    "Diversity and Inclusion", "DEI", "Affirmative Action",
    "Change Management", "Organizational Development", "OD",
    "Employee Engagement", "Employee Retention", "Turnover Reduction",
    "Culture Development", "Workplace Culture", "Employee Experience",
    "Talent Development", "Leadership Development", "Competency Modeling",
    "Career Pathing", "High Potential Programs", "Nine Box Grid",
    "Performance Appraisals", "360 Feedback", "OKRs", "KPIs",
    "HR Analytics", "People Analytics", "Workforce Analytics",
    "Turnover Analysis", "Retention Analytics", "Compensation Analysis",
    "Headcount Planning", "Workforce Modeling", "Predictive Analytics",
    "Technical Recruiting", "Executive Search", "Campus Recruitment",
    "Employer Branding", "Candidate Experience", "Interviewing Techniques",
    "Behavioral Interviewing", "Structured Interviews", "Assessment Centers",
    "Conflict Resolution", "Mediation", "Disciplinary Actions",
    "Grievance Handling", "Workplace Investigations", "Terminations",
    "Exit Interviews", "Stay Interviews", "Employee Surveys",
    "Salary Benchmarking", "Pay Equity Analysis", "Variable Pay",
    "Incentive Programs", "Total Rewards", "Benefits Design",
    "Retirement Plans", "Health Benefits", "Wellness Programs",
    "L&D", "Training Needs Analysis", "Instructional Design",
    "eLearning", "LMS", "Leadership Training", "Soft Skills Training",
    "Coaching", "Mentoring Programs", "Succession Planning"
]
pattern = '|'.join([re.escape(k) for k in hr_keywords])

# Filter and Sample ESCO skills
esco_df_ce_related = esco_df_full[
    esco_df_full['preferredLabel'].str.contains(pattern, case=False, na=False, regex=True) |
    esco_df_full['altLabels'].astype(str).str.contains(pattern, case=False, na=False, regex=True)
].copy()

n_samples_for_prompt = 50
num_available = len(esco_df_ce_related)
if num_available == 0:
    print("WARNING: No Data Scientist related ESCO skills found. ESCO sample will be empty.")
    esco_sample_skills = []
else:
    num_to_sample = min(n_samples_for_prompt, num_available)
    if num_to_sample < n_samples_for_prompt:
        print(f"WARNING: Filtered CE skills ({num_available}) < desired ({n_samples_for_prompt}). Sampling {num_to_sample}.")
    esco_sample_skills = esco_df_ce_related['preferredLabel'].sample(n=num_to_sample, random_state=42).tolist()

esco_skills_string = ", ".join(esco_sample_skills)

#  Few-Shot Examples
few_shot_examples_hr = f"""
Example 1:
Resume: \"\"\"
HR Director - Talent & Organizational Development
Experience:
- Led full-cycle recruitment for 500+ technical and non-technical roles annually
- Implemented Workday HCM to automate onboarding and performance management
- Designed leadership development programs that reduced turnover by 30%
- Conducted pay equity analyses to ensure compliance with OFCCP regulations
\"\"\"
Skills: Talent Acquisition, Workday, Leadership Development, Turnover Reduction, Compensation Analysis, OFCCP Compliance, Performance Management, Onboarding

Example 2:
Resume: \"\"\"
HR Business Partner - Tech Startup
Experience:
- Scaled HR operations from 50 to 500 employees through IPO
- Built DEI initiatives that improved underrepresented group hiring by 40%
- Resolved complex employee relations issues including harassment investigations
- Partnered with engineering leaders to create competitive equity compensation plans
\"\"\"
Skills: HR Scaling, DEI Programs, Employee Relations, Investigations, Startup HR, Equity Compensation, IPO Readiness, HR Business Partnering

Example 3:
Resume: \"\"\"
Recruiting Manager - Global Talent Acquisition
Experience:
- Managed ATS (Greenhouse) supporting 10,000+ annual applications
- Trained hiring managers on behavioral interviewing techniques
- Reduced time-to-hire from 45 to 28 days through process optimization
- Launched employer branding campaign that increased qualified applicants by 60%
\"\"\"
Skills: Technical Recruiting, Greenhouse ATS, Behavioral Interviewing, Hiring Metrics, Employer Branding, Recruitment Marketing, Process Improvement
"""

#  Skill Extraction Function for LLM+KG
LLAMA_MODEL_ID = "meta-llama/Llama-3.3-70B-Instruct"

def extract_skills_llama3_sambanova_kg(resume_text, esco_context_skills_str, few_shot_examples_str, client):
    MAX_RESUME_CHARS_FOR_LLM = 4000
    resume_text_for_llm = resume_text[:MAX_RESUME_CHARS_FOR_LLM] + "..." if len(resume_text) > MAX_RESUME_CHARS_FOR_LLM else resume_text

    system_prompt_kg = (
        "You are a professional resume parser. "
        "Extract all technical skills from the resume below. "
        "Only output a clean, comma-separated list. No extra words or introductory phrases."
        f"\n\n{few_shot_examples_str}\n\n"
        "For guidance and inspiration, consider skills that are closely related to or are present in the following list of relevant ESCO skills::"
        f"[{esco_context_skills_str}]."
    )

    messages = [
        {"role": "system", "content": system_prompt_kg},
        {"role": "user", "content": f"Resume: \"{resume_text_for_llm}\"\n\nSkills:"}
    ]

    completion = client.chat.completions.create(
        model=LLAMA_MODEL_ID,
        messages=messages,
        temperature=0.2,
        max_tokens=200,
    )
    return completion.choices[0].message.content

df_resumes_for_extraction["extracted_skills_raw_kg"] = df_resumes_for_extraction[RESUME_TEXT_COLUMN].progress_apply(
    lambda x: extract_skills_llama3_sambanova_kg(x, esco_skills_string, few_shot_examples_hr, client)
)

output_path_kg = "extracted_skills_kg_output.csv"
df_resumes_for_extraction.to_csv(output_path_kg, index=False)

Extracting Skills (meta-llama/Llama-3.3-70B-Instruct via SambaNova):   0%|          | 0/360 [00:00<?, ?it/s]

In [184]:
df = pd.read_csv("extracted_skills_kg_output.csv")
print(df.head())

          Category                                               Text  \
0  Human Resources  amelia quinton 9835 water street lake zurich i...   
1  Human Resources  sunidhi badruka mobile xxxxxxxxxx email pbadru...   
2  Human Resources  robert wales dubai united arab emirates 050 00...   
3  Human Resources  jane sample 123 main street edmonton ab t5k 2y...   
4  Human Resources  james h maxwell sphr 1876 nightvale path carso...   

                                          Text_Clean  \
0  amelia quinton water street lake zurich il aqu...   
1  sunidhi badruka mobile xxxxxxxxxx email pbadru...   
2  robert wales dubai united arab emirates mobile...   
3  jane sample main street edmonton ab t k y info...   
4  james h maxwell sphr nightvale path carson il ...   

                             extracted_skills_raw_kg  
0  Human Resources, Staff Training, Employee Deve...  
1  Human Resource Management, Recruitment, Sourci...  
2  HR Consulting, Human Capital Management, Compe...  
3  M

### Mapping skills to ESCO

In [185]:
df_mapping = pd.read_csv("extracted_skills_kg_output.csv")
esco_df = pd.read_csv("skills_en.csv")[['preferredLabel', 'altLabels']].copy()

# Clean LLM Output Function
def clean_extracted_skills(text):
    """Cleans raw LLM text output into a set of normalized skill phrases."""
    if not isinstance(text, str) or not text.strip():
        return set()

    text = text.lower().replace(';', ',').replace(' and ', ', ')
    raw_skill_candidates = re.split(r',\s*|\s*,\s*', text)

    return {
        re.sub(r'[^a-z0-9\s\.\+#-]+', '', s).strip()
        for s in raw_skill_candidates
        if s.strip() and len(s.strip()) > 1 and not s.strip().isdigit()
    }

# Apply cleaning
df_mapping['extracted_skills_set'] = df_mapping['extracted_skills_raw_kg'].apply(clean_extracted_skills)

# Pre-process ESCO data for faster lookups
esco_preferred_lower_list = esco_df['preferredLabel'].str.lower().tolist()
pref_map = {label.lower(): label for label in esco_df['preferredLabel'].unique()}

alt_map = {
    alt.strip(): row['preferredLabel']
    for _, row in esco_df.iterrows()
    if isinstance(row['altLabels'], str)
    for alt in row['altLabels'].lower().replace('"', '').split('|')
    if alt.strip()
}

# ESCO Mapping Function
def map_to_esco(skill_name, pref_map_ref, alt_map_ref, esco_lower_list_ref):
    """Maps an extracted skill to an ESCO preferredLabel using exact/fuzzy matching."""
    skill = str(skill_name).lower().strip()
    if not skill:
        return None

    # Exact match (preferred or alternative)
    if skill in pref_map_ref: return pref_map_ref[skill]
    if skill in alt_map_ref: return alt_map_ref[skill]

    # Fuzzy match
    fuzzy_match_result = process.extractOne(skill, esco_lower_list_ref, scorer=fuzz.token_set_ratio)
    if fuzzy_match_result and fuzzy_match_result[1] >= 60: # score >= 60
        # Retrieve original preferredLabel case
        original_label_row = esco_df[esco_df['preferredLabel'].str.lower() == fuzzy_match_result[0]]
        if not original_label_row.empty:
            return original_label_row['preferredLabel'].iloc[0]
    return None

# Apply ESCO Mapping
df_mapping["mapped_skills_str"] = df_mapping['extracted_skills_set'].progress_apply(
    lambda s: ", ".join(sorted(list(filter(None, [map_to_esco(x, pref_map, alt_map, esco_preferred_lower_list) for x in s]))))
)

output_path = "mapped_skills_kg_output.csv"

columns_to_save_kg = [
    'Unnamed: 0', 'Category', 'Text_Clean', 'extracted_skills_raw_kg',
    'original_extracted_skills_for_eval_kg', 'mapped_skills_str'
]

df_mapping['original_extracted_skills_for_eval_kg'] = df_mapping['extracted_skills_set'].apply(lambda s: str(list(s)))

existing_columns_to_save_kg = [col for col in columns_to_save_kg if col in df_mapping.columns]
df_mapping[existing_columns_to_save_kg].to_csv(output_path, index=False)

print(f"Mapped skills saved to {output_path}")

Extracting Skills (meta-llama/Llama-3.3-70B-Instruct via SambaNova):   0%|          | 0/360 [00:00<?, ?it/s]

Mapped skills saved to mapped_skills_kg_output.csv


In [186]:
df = pd.read_csv("mapped_skills_kg_output.csv")
df.head()

,Category,Text_Clean,extracted_skills_raw_kg,original_extracted_skills_for_eval_kg,mapped_skills_str
0,Human Resources,amelia quinton water street lake zurich il aqu...,"Human Resources, Staff Training, Employee Deve...","['microsoft access', 'employee records', 'empl...","Agile development, Agile development, IPC stan..."
1,Human Resources,sunidhi badruka mobile xxxxxxxxxx email pbadru...,"Human Resource Management, Recruitment, Sourci...","['campus placements', 'payroll management', 't...",adopt ways to foster biodiversity and animal w...
2,Human Resources,robert wales dubai united arab emirates mobile...,"HR Consulting, Human Capital Management, Compe...","['job description', 'competency matrices', 'co...","Agile development, adhere to organisational gu..."
3,Human Resources,jane sample main street edmonton ab t k y info...,"MS Word, Excel, PowerPoint, Proprietary Softwa...","['administrative support', 'customer service',...","Agile development, CAM software, CAM software,..."
4,Human Resources,james h maxwell sphr nightvale path carson il ...,"Human Resources, Strategic Planning, Talent Ac...","['diversity', 'talent management', 'employee r...","STAF, analyse issues, animal acquisition, anim..."


### Merge Expected ESCO Skills for LLM+KG Evaluation

In [187]:
df_map = pd.read_csv("mapped_skills_kg_output.csv")
df_exp = pd.read_csv("expected_esco_skills.csv")

#  Clean the category names for consistent merging
df_map['Category'] = df_map['Category'].str.lower().str.strip()
df_exp['category'] = df_exp['category'].str.lower().str.strip()

#  Get the expected ESCO skills for Human Resources
human_resource_skills = df_exp[df_exp['category'] == 'human resources manager']['expected_esco_skills'].iloc[0]

#  Add expected skills column to all Data Science rows
df_map['expected_esco_skills'] = df_map['Category'].apply(
    lambda x: human_resource_skills if x == 'human resources' else None
)

# Reorder columns for better readability
final_df = df_map[['Category','expected_esco_skills','extracted_skills_raw_kg',
                'original_extracted_skills_for_eval_kg', 'mapped_skills_str']]

print(final_df.head().to_markdown(index=False))
final_df.to_csv("mapped_skills_kg_with_expected_esco_for_hr.csv", index=False)

| Category        | expected_esco_skills                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

### Performance Evaluation LLM+KG

In [188]:
df = pd.read_csv("mapped_skills_kg_with_expected_esco_for_hr.csv")

def parse_skills(skills_str):
    """Convert skill string to standardized set of skills."""
    if pd.isna(skills_str) or not str(skills_str).strip():
        return set()
    try:
        # Handles list-like strings (e.g., "['skill1', 'skill2']")
        return {str(s).lower().strip() for s in ast.literal_eval(skills_str)}
    except (ValueError, SyntaxError):
        # Fallback for comma-separated strings (e.g., "skill1, skill2")
        return {s.strip().lower() for s in str(skills_str).split(',') if s.strip()}

# Process skill columns
df['expected_skills'] = df['expected_esco_skills'].apply(parse_skills)
df['mapped_skills'] = df['mapped_skills_str'].apply(parse_skills)

def calculate_metrics(gt_set, pred_set):
    """Calculate precision, recall, and F1-score between skill sets."""
    tp = len(pred_set & gt_set)
    fp = len(pred_set - gt_set)
    fn = len(gt_set - pred_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Filter and calculate metrics for Data Scientist
ce_df = df[df['Category'].str.lower() == 'human resources'].copy()

if ce_df.empty:
    print("No human resources records found with expected skills for LLM+KG evaluation.")
else:
    # Calculate metrics
    ce_df[['precision', 'recall', 'f1_score']] = ce_df.apply(
        lambda r: calculate_metrics(r['expected_skills'], r['mapped_skills']),
        axis=1, result_type='expand'
    )

    # Add intersection skills
    ce_df['intersection_skills'] = ce_df.apply(
        lambda r: list(r['mapped_skills'] & r['expected_skills']), axis=1
    )

    display_cols = ['Category', 'intersection_skills', 'precision',
                    'recall', 'f1_score', 'mapped_skills_str', 'expected_esco_skills']
    print("\n--- LLM+KG Model Performance on Human Resource Skills ---")
    print(ce_df[display_cols].head(10).to_markdown(index=False))
    print(f"\nAverage Precision: {ce_df['precision'].mean():.4f}")
    print(f"Average Recall:    {ce_df['recall'].mean():.4f}")
    print(f"Average F1-Score:  {ce_df['f1_score'].mean():.4f}")


--- LLM+KG Model Performance on Human Resource Skills ---
| Category        | intersection_skills                                                                                                                                                                                                               |   precision |     recall |   f1_score | mapped_skills_str                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            